<a href="https://colab.research.google.com/github/snipe02/Teste-Intuitive-Care/blob/main/Teste2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import os

print("INICIANDO PROCESSAMENTO...")
print("-" * 40)

# 1. JUNTAR OS ARQUIVOS
print("1. Lendo os arquivos...")
arquivos = ['1T2025.csv', '2T2025.csv', '3T2025.csv']
todos_dados = []

for i, arquivo in enumerate(arquivos, 1):
    if not os.path.exists(arquivo):
        print(f"ERRO: {arquivo} não encontrado!")
        exit()

    # Tentar ler com diferentes separadores
    try:
        dados = pd.read_csv(arquivo, sep=';')
    except:
        try:
            dados = pd.read_csv(arquivo, sep=',')
        except:
            dados = pd.read_csv(arquivo)  # tentar auto-detect

    # Guardar nomes originais das colunas
    colunas_originais = list(dados.columns)

    # Mostrar quais colunas o arquivo tem
    print(f"\n   Arquivo: {arquivo}")
    print(f"   Colunas encontradas: {colunas_originais}")

    # Normalizar nomes das colunas (minúsculas, sem espaços)
    dados.columns = [col.strip().lower() for col in dados.columns]

    # Mapear nomes normalizados para originais
    mapa_original = {col.strip().lower(): col for col in colunas_originais}

    # Procurar a coluna CNPJ
    cnpj_col = None
    for col in dados.columns:
        if 'cnpj' in col:
            cnpj_col = col
            break

    # Procurar a coluna Razão Social
    razao_col = None
    for col in dados.columns:
        if 'razao' in col or 'social' in col or 'empresa' in col:
            razao_col = col
            break

    # Procurar a coluna Valor
    valor_col = None
    for col in dados.columns:
        if 'valor' in col or 'despesa' in col:
            valor_col = col
            break

    # Renomear colunas para nomes padrão
    if cnpj_col:
        dados = dados.rename(columns={cnpj_col: 'cnpj'})
        print(f"   ✓ Coluna CNPJ: {mapa_original[cnpj_col]} → cnpj")

    if razao_col:
        dados = dados.rename(columns={razao_col: 'razaosocial'})
        print(f"   ✓ Coluna Razão Social: {mapa_original[razao_col]} → razaosocial")

    if valor_col:
        dados = dados.rename(columns={valor_col: 'valordespesas'})
        print(f"   ✓ Coluna Valor: {mapa_original[valor_col]} → valordespesas")

    # Adicionar colunas de trimestre e ano
    dados['Trimestre'] = i
    dados['Ano'] = 2025

    todos_dados.append(dados)
    print(f"   ✓ {len(dados)} registros carregados")

# Juntar tudo
dados_total = pd.concat(todos_dados, ignore_index=True)
print(f"\n✓ Total de registros: {len(dados_total)}")

# Verificar se as colunas necessárias existem
print("\nVerificando colunas...")
colunas_necessarias = ['cnpj', 'razaosocial', 'valordespesas']

for coluna in colunas_necessarias:
    if coluna not in dados_total.columns:
        print(f"\n⚠️  ATENÇÃO: Coluna '{coluna}' não encontrada!")
        print(f"   Colunas disponíveis:")

        # Mostrar colunas atuais com números
        colunas_disponiveis = list(dados_total.columns)
        for idx, col in enumerate(colunas_disponiveis, 1):
            print(f"   {idx}. {col}")

        # Pedir escolha do usuário
        try:
            escolha = input(f"\n   Qual coluna deve ser usada como '{coluna}'? (👉digite o número 1 a  {len(colunas_disponiveis)}): ")
            escolha_idx = int(escolha) - 1

            if 0 <= escolha_idx < len(colunas_disponiveis):
                col_escolhida = colunas_disponiveis[escolha_idx]
                dados_total = dados_total.rename(columns={col_escolhida: coluna})
                print(f"   ✓ '{col_escolhida}' renomeada para '{coluna}'")
            else:
                print(f"   ❌ Número inválido. Usando valor padrão para '{coluna}'...")
                dados_total[coluna] = ""
        except ValueError:
            print(f"   ❌ Entrada inválida. Usando valor padrão para '{coluna}'...")
            dados_total[coluna] = ""
        except Exception as e:
            print(f"   ❌ Erro: {e}. Usando valor padrão para '{coluna}'...")
            dados_total[coluna] = ""

# Converter valores para numérico se necessário
if 'valordespesas' in dados_total.columns:
    try:
        # Tentar converter para numérico
        dados_total['valordespesas'] = pd.to_numeric(dados_total['valordespesas'], errors='coerce')
    except:
        print("⚠️  Não foi possível converter valores para numérico")

# 2. VALIDAR CNPJ
print("\n2. Validando CNPJs...")
if 'cnpj' in dados_total.columns:
    def validar_cnpj_simples(cnpj):
        if pd.isna(cnpj):
            return False
        # Converter para string
        cnpj_str = str(cnpj)
        # Remover caracteres não numéricos
        numeros = re.sub(r'\D', '', cnpj_str)
        return len(numeros) == 14

    dados_total['CNPJ_OK'] = dados_total['cnpj'].apply(validar_cnpj_simples)
    print(f"   ✓ CNPJs válidos: {dados_total['CNPJ_OK'].sum()} de {len(dados_total)}")
else:
    dados_total['CNPJ_OK'] = False
    print("   ⚠️  Coluna CNPJ não disponível")

# 3. VALIDAR VALOR
print("\n3. Validando valores...")
if 'valordespesas' in dados_total.columns:
    # Preencher NaN com 0
    dados_total['valordespesas'] = dados_total['valordespesas'].fillna(0)
    dados_total['VALOR_OK'] = dados_total['valordespesas'] > 0
    print(f"   ✓ Valores positivos: {dados_total['VALOR_OK'].sum()} de {len(dados_total)}")
else:
    dados_total['VALOR_OK'] = False
    print("   ⚠️  Coluna Valor não disponível")

# 4. VALIDAR RAZÃO SOCIAL
print("\n4. Validando razão social...")
if 'razaosocial' in dados_total.columns:
    # Preencher NaN com string vazia
    dados_total['razaosocial'] = dados_total['razaosocial'].fillna('')
    dados_total['RAZAO_OK'] = dados_total['razaosocial'].astype(str).str.strip() != ''
    print(f"   ✓ Razões sociais preenchidas: {dados_total['RAZAO_OK'].sum()} de {len(dados_total)}")
else:
    dados_total['RAZAO_OK'] = False
    print("   ⚠️  Coluna Razão Social não disponível")

# 5. SEPARAR
print("\n5. Separando resultados...")
# Verificar se todas as colunas de validação existem
colunas_validacao = ['CNPJ_OK', 'VALOR_OK', 'RAZAO_OK']
if all(col in dados_total.columns for col in colunas_validacao):
    dados_ok = dados_total[dados_total['CNPJ_OK'] & dados_total['VALOR_OK'] & dados_total['RAZAO_OK']]
    dados_erro = dados_total[~(dados_total['CNPJ_OK'] & dados_total['VALOR_OK'] & dados_total['RAZAO_OK'])]
    print(f"   ✓ Dados corretos: {len(dados_ok)}")
    print(f"   ✓ Dados com erro: {len(dados_erro)}")
else:
    print("   ⚠️  Não foi possível separar os dados - algumas validações não foram realizadas")
    dados_ok = pd.DataFrame()
    dados_erro = dados_total

# 6. SALVAR
print("\n6. Salvando arquivos...")
dados_total.to_csv('todos_dados.csv', index=False, sep=';', encoding='utf-8')
print("   ✓ todos_dados.csv salvo")

if len(dados_ok) > 0:
    dados_ok.to_csv('resultado_ok.csv', index=False, sep=';', encoding='utf-8')
    print("   ✓ resultado_ok.csv salvo")

if len(dados_erro) > 0:
    dados_erro.to_csv('resultado_erro.csv', index=False, sep=';', encoding='utf-8')
    print("   ✓ resultado_erro.csv salvo")

# 7. RESUMO
print("\n" + "=" * 40)
print("RESULTADO FINAL:")
print("=" * 40)
print(f"📊 TOTAL DE REGISTROS: {len(dados_total)}")

if 'CNPJ_OK' in dados_total.columns:
    print(f"📋 CNPJs VÁLIDOS: {dados_total['CNPJ_OK'].sum()}")

if 'VALOR_OK' in dados_total.columns:
    print(f"💰 VALORES POSITIVOS: {dados_total['VALOR_OK'].sum()}")

if 'RAZAO_OK' in dados_total.columns:
    print(f"🏢 RAZÕES SOCIAIS PREENCHIDAS: {dados_total['RAZAO_OK'].sum()}")

if len(dados_ok) > 0:
    print(f"\n✅ DADOS CORRETOS (tudo OK): {len(dados_ok)}")

if len(dados_erro) > 0:
    print(f"❌ DADOS COM ERROS: {len(dados_erro)}")

print(f"\n💾 ARQUIVOS CRIADOS:")
arquivos_criados = []
for arquivo in ['todos_dados.csv', 'resultado_ok.csv', 'resultado_erro.csv']:
    if os.path.exists(arquivo):
        tamanho_kb = os.path.getsize(arquivo) / 1024
        arquivos_criados.append(f"   • {arquivo} ({tamanho_kb:.1f} KB)")

if arquivos_criados:
    print("\n".join(arquivos_criados))
else:
    print("   Nenhum arquivo foi criado")

print("\n✅ PROCESSO CONCLUÍDO!")

INICIANDO PROCESSAMENTO...
----------------------------------------
1. Lendo os arquivos...

   Arquivo: 1T2025.csv
   Colunas encontradas: ['DATA', 'REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO', 'VL_SALDO_INICIAL', 'VL_SALDO_FINAL']
   ✓ 705390 registros carregados

   Arquivo: 2T2025.csv
   Colunas encontradas: ['DATA', 'REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO', 'VL_SALDO_INICIAL', 'VL_SALDO_FINAL']
   ✓ 696591 registros carregados

   Arquivo: 3T2025.csv
   Colunas encontradas: ['DATA', 'REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO', 'VL_SALDO_INICIAL', 'VL_SALDO_FINAL']
   ✓ 711943 registros carregados

✓ Total de registros: 2113924

Verificando colunas...

⚠️  ATENÇÃO: Coluna 'cnpj' não encontrada!
   Colunas disponíveis:
   1. data
   2. reg_ans
   3. cd_conta_contabil
   4. descricao
   5. vl_saldo_inicial
   6. vl_saldo_final
   7. Trimestre
   8. Ano
